In [1]:
from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image

In [2]:
ls

Image VAE.ipynb                 poemfiles/
Study Data.ipynb                poetry_foundation_scraper.py
chromedriver*                   poetry_scraper_improved.py
edited_poems.csv                poetry_scraper_improved_old.py
full_scraper.py                 results/
glove/                          rhyme_dict_full.txt
lyrics_billboard.csv*           rhyme_dict_simple.txt
lyrics_large.csv*               rhyme_phones.txt
lyrics_small.csv*               scraper/
mlcourse/                       scraper.py
poem_VAE.py                     small_poem_set.csv


In [3]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.ToTensor()),
    batch_size=32, shuffle=True)

In [4]:
ls ../data

cifar-10-batches-py/          processed/
cornell movie-dialogs corpus/ raw/


In [5]:
batch_size = 32
log_interval = 10

In [8]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.encode1 = nn.Linear(784, 400)
        self.encode2 = nn.Linear(400, 100)
        self.encode31 = nn.Linear(100, 10)
        self.encode32 = nn.Linear(100, 10)
        self.decode1 = nn.Linear(10, 100)
        self.decode2 = nn.Linear(100, 400)
        self.decode3 = nn.Linear(400, 784)

    def encode(self, x):
        h1 = F.relu(self.encode1(x))
        h2 = F.relu(self.encode2(h1))
        return self.encode31(h2), self.encode32(h2)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return eps.mul(std).add_(mu)

    def decode(self, z):
        h3 = F.relu(self.decode1(z))
        h4 = F.relu(self.decode2(h3))
        return torch.sigmoid(self.decode3(h4))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 784))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar
    

# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction="sum")

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + KLD


def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))
    

def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (data, _) in enumerate(test_loader):
            data = data.to(device)
            recon_batch, mu, logvar = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar).item()
            if i == 0:
                n = min(data.size(0), 8)
                comparison = torch.cat([data[:n],
                                      recon_batch.view(batch_size, 1, 28, 28)[:n]])
                save_image(comparison.cpu(),
                         'results/reconstruction_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [11]:
device = torch.device("cpu")
model = VAE().to(device)
optimizer = optim.Adam(model.parameters(), lr=2e-4)

epochs = 10
for epoch in range(1, epochs + 1):
    train(epoch)
    test(epoch)
    with torch.no_grad():
        sample = torch.randn(64, 10).to(device)
        sample = model.decode(sample).cpu()
        save_image(sample.view(64, 1, 28, 28),
                   'results/sample_' + str(epoch) + '.png')

Train Epoch: 1 [0/60000 (0%)]	Loss: 544.654419
Train Epoch: 1 [320/60000 (1%)]	Loss: 518.053528
Train Epoch: 1 [640/60000 (1%)]	Loss: 484.653290
Train Epoch: 1 [960/60000 (2%)]	Loss: 395.094452
Train Epoch: 1 [1280/60000 (2%)]	Loss: 290.077484
Train Epoch: 1 [1600/60000 (3%)]	Loss: 245.589462
Train Epoch: 1 [1920/60000 (3%)]	Loss: 243.209015
Train Epoch: 1 [2240/60000 (4%)]	Loss: 229.086670
Train Epoch: 1 [2560/60000 (4%)]	Loss: 237.467743
Train Epoch: 1 [2880/60000 (5%)]	Loss: 226.884521
Train Epoch: 1 [3200/60000 (5%)]	Loss: 221.428833
Train Epoch: 1 [3520/60000 (6%)]	Loss: 218.585892
Train Epoch: 1 [3840/60000 (6%)]	Loss: 222.603058
Train Epoch: 1 [4160/60000 (7%)]	Loss: 209.617004
Train Epoch: 1 [4480/60000 (7%)]	Loss: 218.664093
Train Epoch: 1 [4800/60000 (8%)]	Loss: 209.616959
Train Epoch: 1 [5120/60000 (9%)]	Loss: 204.366943
Train Epoch: 1 [5440/60000 (9%)]	Loss: 214.011108
Train Epoch: 1 [5760/60000 (10%)]	Loss: 211.362625
Train Epoch: 1 [6080/60000 (10%)]	Loss: 212.140488
Trai

Train Epoch: 1 [51200/60000 (85%)]	Loss: 147.625412
Train Epoch: 1 [51520/60000 (86%)]	Loss: 157.491577
Train Epoch: 1 [51840/60000 (86%)]	Loss: 146.578461
Train Epoch: 1 [52160/60000 (87%)]	Loss: 164.050308
Train Epoch: 1 [52480/60000 (87%)]	Loss: 151.657806
Train Epoch: 1 [52800/60000 (88%)]	Loss: 165.677017
Train Epoch: 1 [53120/60000 (89%)]	Loss: 140.111664
Train Epoch: 1 [53440/60000 (89%)]	Loss: 145.568695
Train Epoch: 1 [53760/60000 (90%)]	Loss: 140.691162
Train Epoch: 1 [54080/60000 (90%)]	Loss: 147.443665
Train Epoch: 1 [54400/60000 (91%)]	Loss: 147.417999
Train Epoch: 1 [54720/60000 (91%)]	Loss: 146.369888
Train Epoch: 1 [55040/60000 (92%)]	Loss: 136.636398
Train Epoch: 1 [55360/60000 (92%)]	Loss: 147.659546
Train Epoch: 1 [55680/60000 (93%)]	Loss: 148.757431
Train Epoch: 1 [56000/60000 (93%)]	Loss: 142.973984
Train Epoch: 1 [56320/60000 (94%)]	Loss: 154.896591
Train Epoch: 1 [56640/60000 (94%)]	Loss: 138.925446
Train Epoch: 1 [56960/60000 (95%)]	Loss: 148.482101
Train Epoch:

Train Epoch: 2 [41920/60000 (70%)]	Loss: 133.095291
Train Epoch: 2 [42240/60000 (70%)]	Loss: 133.520935
Train Epoch: 2 [42560/60000 (71%)]	Loss: 132.036453
Train Epoch: 2 [42880/60000 (71%)]	Loss: 122.419640
Train Epoch: 2 [43200/60000 (72%)]	Loss: 140.259277
Train Epoch: 2 [43520/60000 (73%)]	Loss: 120.931122
Train Epoch: 2 [43840/60000 (73%)]	Loss: 128.490356
Train Epoch: 2 [44160/60000 (74%)]	Loss: 133.120193
Train Epoch: 2 [44480/60000 (74%)]	Loss: 137.761261
Train Epoch: 2 [44800/60000 (75%)]	Loss: 133.586716
Train Epoch: 2 [45120/60000 (75%)]	Loss: 136.818726
Train Epoch: 2 [45440/60000 (76%)]	Loss: 131.076141
Train Epoch: 2 [45760/60000 (76%)]	Loss: 139.124664
Train Epoch: 2 [46080/60000 (77%)]	Loss: 135.457947
Train Epoch: 2 [46400/60000 (77%)]	Loss: 127.648766
Train Epoch: 2 [46720/60000 (78%)]	Loss: 139.488037
Train Epoch: 2 [47040/60000 (78%)]	Loss: 134.073975
Train Epoch: 2 [47360/60000 (79%)]	Loss: 133.715714
Train Epoch: 2 [47680/60000 (79%)]	Loss: 134.060699
Train Epoch:

Train Epoch: 3 [32640/60000 (54%)]	Loss: 131.354446
Train Epoch: 3 [32960/60000 (55%)]	Loss: 122.527191
Train Epoch: 3 [33280/60000 (55%)]	Loss: 119.802719
Train Epoch: 3 [33600/60000 (56%)]	Loss: 128.104584
Train Epoch: 3 [33920/60000 (57%)]	Loss: 128.276184
Train Epoch: 3 [34240/60000 (57%)]	Loss: 117.063881
Train Epoch: 3 [34560/60000 (58%)]	Loss: 123.599792
Train Epoch: 3 [34880/60000 (58%)]	Loss: 113.772682
Train Epoch: 3 [35200/60000 (59%)]	Loss: 127.401726
Train Epoch: 3 [35520/60000 (59%)]	Loss: 132.179153
Train Epoch: 3 [35840/60000 (60%)]	Loss: 129.292374
Train Epoch: 3 [36160/60000 (60%)]	Loss: 125.910538
Train Epoch: 3 [36480/60000 (61%)]	Loss: 128.210541
Train Epoch: 3 [36800/60000 (61%)]	Loss: 141.794449
Train Epoch: 3 [37120/60000 (62%)]	Loss: 132.964478
Train Epoch: 3 [37440/60000 (62%)]	Loss: 114.689789
Train Epoch: 3 [37760/60000 (63%)]	Loss: 124.724007
Train Epoch: 3 [38080/60000 (63%)]	Loss: 125.936600
Train Epoch: 3 [38400/60000 (64%)]	Loss: 117.359978
Train Epoch:

Train Epoch: 4 [23040/60000 (38%)]	Loss: 107.808228
Train Epoch: 4 [23360/60000 (39%)]	Loss: 129.328430
Train Epoch: 4 [23680/60000 (39%)]	Loss: 105.338661
Train Epoch: 4 [24000/60000 (40%)]	Loss: 120.007614
Train Epoch: 4 [24320/60000 (41%)]	Loss: 109.733276
Train Epoch: 4 [24640/60000 (41%)]	Loss: 112.451492
Train Epoch: 4 [24960/60000 (42%)]	Loss: 112.442818
Train Epoch: 4 [25280/60000 (42%)]	Loss: 118.033653
Train Epoch: 4 [25600/60000 (43%)]	Loss: 111.027176
Train Epoch: 4 [25920/60000 (43%)]	Loss: 133.540985
Train Epoch: 4 [26240/60000 (44%)]	Loss: 117.507591
Train Epoch: 4 [26560/60000 (44%)]	Loss: 118.122543
Train Epoch: 4 [26880/60000 (45%)]	Loss: 118.105972
Train Epoch: 4 [27200/60000 (45%)]	Loss: 114.773773
Train Epoch: 4 [27520/60000 (46%)]	Loss: 119.431152
Train Epoch: 4 [27840/60000 (46%)]	Loss: 118.509346
Train Epoch: 4 [28160/60000 (47%)]	Loss: 118.941513
Train Epoch: 4 [28480/60000 (47%)]	Loss: 119.270020
Train Epoch: 4 [28800/60000 (48%)]	Loss: 119.844559
Train Epoch:

...

Train Epoch: 6 [55040/60000 (92%)]	Loss: 123.042755
Train Epoch: 6 [55360/60000 (92%)]	Loss: 110.920418
Train Epoch: 6 [55680/60000 (93%)]	Loss: 120.177605
Train Epoch: 6 [56000/60000 (93%)]	Loss: 115.396935
Train Epoch: 6 [56320/60000 (94%)]	Loss: 115.875183
Train Epoch: 6 [56640/60000 (94%)]	Loss: 114.943901
Train Epoch: 6 [56960/60000 (95%)]	Loss: 112.999680
Train Epoch: 6 [57280/60000 (95%)]	Loss: 110.813370
Train Epoch: 6 [57600/60000 (96%)]	Loss: 107.862793
Train Epoch: 6 [57920/60000 (97%)]	Loss: 104.925354
Train Epoch: 6 [58240/60000 (97%)]	Loss: 107.673622
Train Epoch: 6 [58560/60000 (98%)]	Loss: 115.850525
Train Epoch: 6 [58880/60000 (98%)]	Loss: 110.624893
Train Epoch: 6 [59200/60000 (99%)]	Loss: 112.074905
Train Epoch: 6 [59520/60000 (99%)]	Loss: 110.342522
Train Epoch: 6 [59840/60000 (100%)]	Loss: 110.303146
====> Epoch: 6 Average loss: 114.9139
====> Test set loss: 113.3354
Train Epoch: 7 [0/60000 (0%)]	Loss: 117.784348
Train Epoch: 7 [320/60000 (1%)]	Loss: 109.071091
Tra

Train Epoch: 7 [45440/60000 (76%)]	Loss: 111.168671
Train Epoch: 7 [45760/60000 (76%)]	Loss: 111.147804
Train Epoch: 7 [46080/60000 (77%)]	Loss: 115.856232
Train Epoch: 7 [46400/60000 (77%)]	Loss: 121.635612
Train Epoch: 7 [46720/60000 (78%)]	Loss: 113.362022
Train Epoch: 7 [47040/60000 (78%)]	Loss: 105.227371
Train Epoch: 7 [47360/60000 (79%)]	Loss: 108.142700
Train Epoch: 7 [47680/60000 (79%)]	Loss: 103.454704
Train Epoch: 7 [48000/60000 (80%)]	Loss: 110.263107
Train Epoch: 7 [48320/60000 (81%)]	Loss: 113.019989
Train Epoch: 7 [48640/60000 (81%)]	Loss: 108.841995
Train Epoch: 7 [48960/60000 (82%)]	Loss: 106.249420
Train Epoch: 7 [49280/60000 (82%)]	Loss: 129.397720
Train Epoch: 7 [49600/60000 (83%)]	Loss: 112.007660
Train Epoch: 7 [49920/60000 (83%)]	Loss: 115.005112
Train Epoch: 7 [50240/60000 (84%)]	Loss: 112.809448
Train Epoch: 7 [50560/60000 (84%)]	Loss: 106.820175
Train Epoch: 7 [50880/60000 (85%)]	Loss: 118.325546
Train Epoch: 7 [51200/60000 (85%)]	Loss: 105.863815
Train Epoch:

Train Epoch: 8 [35840/60000 (60%)]	Loss: 114.964378
Train Epoch: 8 [36160/60000 (60%)]	Loss: 111.325241
Train Epoch: 8 [36480/60000 (61%)]	Loss: 109.820793
Train Epoch: 8 [36800/60000 (61%)]	Loss: 109.748756
Train Epoch: 8 [37120/60000 (62%)]	Loss: 103.649086
Train Epoch: 8 [37440/60000 (62%)]	Loss: 119.165421
Train Epoch: 8 [37760/60000 (63%)]	Loss: 111.704079
Train Epoch: 8 [38080/60000 (63%)]	Loss: 112.775818
Train Epoch: 8 [38400/60000 (64%)]	Loss: 112.368347
Train Epoch: 8 [38720/60000 (65%)]	Loss: 114.724953
Train Epoch: 8 [39040/60000 (65%)]	Loss: 114.618561
Train Epoch: 8 [39360/60000 (66%)]	Loss: 108.061882
Train Epoch: 8 [39680/60000 (66%)]	Loss: 121.554070
Train Epoch: 8 [40000/60000 (67%)]	Loss: 107.122543
Train Epoch: 8 [40320/60000 (67%)]	Loss: 114.658806
Train Epoch: 8 [40640/60000 (68%)]	Loss: 116.113609
Train Epoch: 8 [40960/60000 (68%)]	Loss: 105.073402
Train Epoch: 8 [41280/60000 (69%)]	Loss: 111.027512
Train Epoch: 8 [41600/60000 (69%)]	Loss: 104.194733
Train Epoch:

Train Epoch: 9 [26240/60000 (44%)]	Loss: 109.719734
Train Epoch: 9 [26560/60000 (44%)]	Loss: 109.300049
Train Epoch: 9 [26880/60000 (45%)]	Loss: 115.706757
Train Epoch: 9 [27200/60000 (45%)]	Loss: 115.597221
Train Epoch: 9 [27520/60000 (46%)]	Loss: 116.320267
Train Epoch: 9 [27840/60000 (46%)]	Loss: 113.236755
Train Epoch: 9 [28160/60000 (47%)]	Loss: 115.804802
Train Epoch: 9 [28480/60000 (47%)]	Loss: 108.656395
Train Epoch: 9 [28800/60000 (48%)]	Loss: 118.644554
Train Epoch: 9 [29120/60000 (49%)]	Loss: 104.286987
Train Epoch: 9 [29440/60000 (49%)]	Loss: 99.305618
Train Epoch: 9 [29760/60000 (50%)]	Loss: 107.692413
Train Epoch: 9 [30080/60000 (50%)]	Loss: 115.887680
Train Epoch: 9 [30400/60000 (51%)]	Loss: 115.549850
Train Epoch: 9 [30720/60000 (51%)]	Loss: 116.086250
Train Epoch: 9 [31040/60000 (52%)]	Loss: 106.067734
Train Epoch: 9 [31360/60000 (52%)]	Loss: 107.144348
Train Epoch: 9 [31680/60000 (53%)]	Loss: 114.315521
Train Epoch: 9 [32000/60000 (53%)]	Loss: 108.273796
Train Epoch: 

Train Epoch: 10 [16640/60000 (28%)]	Loss: 113.053864
Train Epoch: 10 [16960/60000 (28%)]	Loss: 101.721657
Train Epoch: 10 [17280/60000 (29%)]	Loss: 105.703033
Train Epoch: 10 [17600/60000 (29%)]	Loss: 112.668785
Train Epoch: 10 [17920/60000 (30%)]	Loss: 118.350929
Train Epoch: 10 [18240/60000 (30%)]	Loss: 116.604630
Train Epoch: 10 [18560/60000 (31%)]	Loss: 102.542320
Train Epoch: 10 [18880/60000 (31%)]	Loss: 109.724869
Train Epoch: 10 [19200/60000 (32%)]	Loss: 109.970840
Train Epoch: 10 [19520/60000 (33%)]	Loss: 114.917450
Train Epoch: 10 [19840/60000 (33%)]	Loss: 107.297768
Train Epoch: 10 [20160/60000 (34%)]	Loss: 102.945869
Train Epoch: 10 [20480/60000 (34%)]	Loss: 104.387100
Train Epoch: 10 [20800/60000 (35%)]	Loss: 107.415092
Train Epoch: 10 [21120/60000 (35%)]	Loss: 102.596466
Train Epoch: 10 [21440/60000 (36%)]	Loss: 116.061478
Train Epoch: 10 [21760/60000 (36%)]	Loss: 109.972076
Train Epoch: 10 [22080/60000 (37%)]	Loss: 111.421173
Train Epoch: 10 [22400/60000 (37%)]	Loss: 111.